In [10]:
using LinearAlgebra
using Convex
using SCS

"The Goemans-Williamson algorithm for the MAXCUT problem."
"https://github.com/ericproffitt/maxcut.jl"

function maxcut(W::Matrix{<:Real}; iter::Int=100, tol::Real=1e-1)
    "Partition a graph into two disjoint sets such that the sum of the edge weights
    which cross the partition is as large as possible (known to be NP-hard)."

    "A cut of a graph can be produced by assigning either 1 or -1 to each vertex. The Goemans-Williamson 
    algorithm relaxes this binary condition to allow for vector assignments drawn from the (n-1)-sphere
    (choosing an n-1 dimensional space will ensure seperability). This relaxation can then be written as 
    an semidefinite program. Once the optimal vector assignments are found, origin centered hyperplanes are generated and 
    their corresponding cuts evaluated. After 'iter' trials, or when the desired tolerance is reached,
    the hyperplane with the highest corresponding binary cut is used to partition the vertices."
    
    "W:     Adjacency matrix."
    "tol:   Maximum acceptable distance between a cut and the MAXCUT upper bound."
    "iter:  Maximum number of hyperplane iterations before a cut is chosen."

    LinearAlgebra.checksquare(W)
    issymmetric(W)					|| throw(ArgumentError("Adjacency matrix must be symmetric."))
    all(W .>= 0)					|| throw(ArgumentError("Adjacency matrix must be nonnegative."))
    all(iszero.(diag(W)))			|| throw(ArgumentError("Diagonal of adjacency matrix must be zero (no self loops)."))
    (tol >= 0)						|| throw(ArgumnetError("The tolerance must be nonnegative."))
    (iter > 0)						|| throw(ArgumnetError("The number of iterations must be a positive integer."))

    "This is the standard SDP Relaxation of the MAXCUT problem, a reference can be found at
    http://www.sfu.ca/~mdevos/notes/semidef/GW.pdf."
    k = size(W, 1)
    S = Semidefinite(k)

    expr = dot(W, S)
    constr = [S[i,i] == 1.0 for i in 1:k]
    problem = minimize(expr, constr...)
    solve!(problem, SCSSolver(verbose=0))

    ### Ensure symmetric positive-definite.
    A = 0.5 * (S.value + S.value')
    A += (max(0, -eigmin(A)) + 1e-14) * Matrix(I, size(A, 1), size(A, 1))
    
    X = Matrix(cholesky(A))

    ### A non-trivial upper bound on MAXCUT.
    upperbound = (sum(W) - dot(W, S.value)) / 4 

    "Random origin-centered hyperplanes, generated to produce partitions of the graph."
    max_cut = 0
    max_partition = nothing

    for i in 1:iter
        gweval = X' * randn(k)
        partition = (findall(gweval .>= 0), findall(gweval .< 0))
        cut = sum(W[partition...])

        if cut > max_cut
            max_partition = partition
            max_cut = cut
        end

        (upperbound - max_cut < tol) && break
        (i == iter) && println("Max iterations reached.")
    end
    return round(max_cut, digits=3), max_partition
end

maxcut (generic function with 1 method)

In [6]:
using Pkg
Pkg.add("SCS")

 Resolving package versions...
 Installed IniFile ──────────── v0.5.0
 Installed JSONSchema ───────── v0.2.0
 Installed CodecZlib ────────── v0.6.0
 Installed CodecBzip2 ───────── v0.6.0
 Installed SCS ──────────────── v0.6.4
 Installed HTTP ─────────────── v0.8.8
 Installed TranscodingStreams ─ v0.9.5
 Installed MathOptInterface ─── v0.9.10
 Installed MutableArithmetics ─ v0.2.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c946c3f1] + SCS v0.6.4
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [523fee87] + CodecBzip2 v0.6.0
  [944b1d66] + CodecZlib v0.6.0
  [cd3eb016] + HTTP v0.8.8
  [83e8ac13] + IniFile v0.5.0
  [7d188eb4] + JSONSchema v0.2.0
  [b8f27783] + MathOptInterface v0.9.10
  [d8a4904e] + MutableArithmetics v0.2.2
  [c946c3f1] + SCS v0.6.4
  [3bb67fe8] + TranscodingStreams v0.9.5
  Building CodecZlib ─→ `~/.julia/packages/CodecZlib/5t9zO/deps/build.log`
  Building CodecBzip2 → `~/.julia/packages/CodecBzip2/T5yr7/deps/build.log`
  Building SCS ───────→ `~/.ju

In [4]:
using Pkg
Pkg.add("Convex")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Compat ───────────── v2.2.0
 Installed MathProgBase ─────── v0.7.7
 Installed BenchmarkTools ───── v0.4.3
 Installed AbstractTrees ────── v0.2.1
 Installed OrderedCollections ─ v1.1.0
 Installed Convex ───────────── v0.12.6
  Updating `~/.julia/environments/v1.0/Project.toml`
  [f65535da] + Convex v0.12.6
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [1520ce14] + AbstractTrees v0.2.1
  [6e4b80f9] + BenchmarkTools v0.4.3
  [34da2185] + Compat v2.2.0
  [f65535da] + Convex v0.12.6
  [fdba3010] + MathProgBase v0.7.7
  [bac558e1] + OrderedCollections v1.1.0
  [8bb1440f] + DelimitedFiles 
  [1a1011a3] + SharedArrays 
  [2f01184e] + SparseArrays 
  [10745b16] + Statistics 
